In [43]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

In [44]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [ ]:
import os
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

from dotenv import load_dotenv
import os

load_dotenv()  # Load variables from .env

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


# Step 1: Load the web document
loader = WebBaseLoader(
    web_paths=["https://lilianweng.github.io/posts/2023-06-23-agent/"],
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(class_=("post-content", "post-title", "post-header"))
    )
)
docs = loader.load()

# Step 2: Split document into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(docs)

# Step 3: Generate embeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Step 4: Store in FAISS vector store
vector_store = FAISS.from_documents(chunks, embeddings)

# Step 5: Similarity Search
query = "What is Machine learning"
results = vector_store.similarity_search(query, k=3)

# Step 6: Prepare context for LLM
context = "\n\n".join([doc.page_content for doc in results])

# Step 7: Set up Gemini as LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",  # or "gemini-1.5-flash" for a faster, cheaper option
    google_api_key=os.environ["GOOGLE_API_KEY"]
)

# Step 8: Prepare prompt
prompt_template = ChatPromptTemplate.from_template("""
You are a helpful AI assistant. Use the context below to answer the user's question.

Context:
{context}

Question:
{question}

Answer:""")

prompt = prompt_template.invoke({"context": context, "question": query})

# Step 9: Call the LLM
response = llm.invoke(prompt)
print("\n--- FINAL ANSWER ---\n")
print(response.content)



--- FINAL ANSWER ---

Machine learning is not explicitly defined in the given context. However, the context does mention several applications of machine learning within the framework of LLM-powered autonomous agents, such as using LLMs as routers in MRKL systems, leveraging them for scientific discovery, and employing them for tasks like designing experiments and using tools.
